## **Model Evaluations**
### **Objectives**

This project involves a complete script/notebook that processes a set of sample images for object detection, annotating each image with bounding boxes, labels, and confidence scores for all detected objects. The project includes the following key components:
* *Image Processing:* The script reads our set of sample images to be used for object detection.
    * Note: For now the script runs on an experimental sample set, but more to come...

* *Object Detection:* Multiple pre-trained models are utilized to detect objects within the images.

    * Faster R-CNN
    * FCOS
    * RetinaNet
    * SSD
    * SSDlite

* *Visualization:* The results of the object detection are visualized with a side-by-side comparison of the annotated images from each model. Comparisions look at the top 5 predictions from each model and their respective bounding boxes on the same image.






#### **Imports**

Note: this is a lot imports. If this is the first time that you running the notebook, it may take a while.

In [81]:
# Importing Utilities
import os
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes, make_grid
from torchvision.transforms.functional import to_pil_image
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import regex as re
import matplotlib.pyplot as plt
import torchvision


# Importing the Models and their respective weights
from torchvision.models.detection import (
    # Faster R-CNN
    fasterrcnn_resnet50_fpn_v2,
    FasterRCNN_ResNet50_FPN_V2_Weights,
    # # FCOS
    fcos_resnet50_fpn,
    FCOS_ResNet50_FPN_Weights,
    # RetinaNet
    retinanet_resnet50_fpn_v2,
    RetinaNet_ResNet50_FPN_V2_Weights,
    # SSD
    ssd300_vgg16,
    SSD300_VGG16_Weights,
    # SSDlite
    ssdlite320_mobilenet_v3_large,
    SSDLite320_MobileNet_V3_Large_Weights,
)

#### **save_image_with_title function**
This function creates an output directory and subdirectories. In addition, it saves the images to a corresponding model subdirectories using the following:
* *title*: The title of the Model's Output: **Top Detections of the {model_name}: {score_len} detections at {threshold}**
* *output_path*: The main directory where you want to save the output image.
* *model_name*: The name of the model, used to create a subdirectory. 
* *image_path*: The original image path. The function extracts the base name of the image.
* *im*: Model modified or unmodified PIL Image.

In [82]:
def save_image_with_title(title, image, output_path, model_name, image_path):
    # Convert the PIL image to a format that Matplotlib can handle
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.title(title, fontsize=15)
    plt.axis("off")

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # creating the model's output directory
    model_output_directory = os.path.join(output_path, model_name)
    if not os.path.exists(model_output_directory):
        os.makedirs(model_output_directory)
    base_name = os.path.basename(image_path)
    output_file = os.path.join(model_output_directory, base_name)

    # Save the figure
    plt.savefig(output_file, bbox_inches="tight", pad_inches=0.2)
    plt.close()
    print(f"Model Output saved to {output_file}")

#### **object_detection function**
This function generates the input model, processes the input image using model weights, and outputs the a visualization of the model's prediction (as annotated bounding boxes) to an output path.
* *model*: model from *torchvision.models.detection*. This should already be imported.
* *weights*: weights that correspond to model. Review torchvision model documentation.
* *image_path*: original image path.
* *output_path*: The main directory where you want to save the output image. DEFAULT= "evaluation/outputs".
* *threshold*: model probability score threshold. DEFAULT= 0.90

In [83]:
def object_detection(
    model, weights, image_path, output_path="evaluation/outputs", threshold=0.90
):
    img = read_image(image_path)
    # Step 1: Initialize model with the best available weights
    weights = weights.DEFAULT
    model_name = model.__name__

    model = model(weights=weights)
    model.eval()

    # Step 2: Initialize the inference transforms
    preprocess = weights.transforms()

    # Step 3: Apply inference preprocessing transforms
    batch = [preprocess(img)]

    # Step 4: Use the model and visualize the prediction
    prediction = model(batch)[0]

    # Extracting the len of Index of the scores that meet the threshold value:
    score_len = (prediction["scores"] >= threshold).sum().item()
    # Limits the scores at the threshold to just the top 5
    if score_len >= 5:
        score_len = 5
    else:
        pass

    # Step 5: Visualizing Model predictions onto image.

    # Annotation features
    title = f"Evaluating Top Detections\n{model_name}\n{score_len} detections at {threshold}"
    fill = (170, 51, 106)
    font_path = os.path.abspath("fonts/OpenSans-Regular.ttf")
    font = ImageFont.truetype(font_path, 30)

    if score_len == 0:  # If there are no predictions at the threshold, just o
        im = to_pil_image(img)

    else:  # If there are predictions, generate annotated visualization and save to corresponding output directory
        labels = [
            weights.meta["categories"][i] for i in prediction["labels"][:score_len]
        ]
        scores = prediction["scores"][:score_len]
        labels_with_scores = [
            f"{label} {score:.2f}" for label, score in zip(labels, scores)
        ]

        box = draw_bounding_boxes(
            img,
            boxes=prediction["boxes"][:score_len],
            labels=labels_with_scores,
            colors="red",
            width=4,
            font=font_path,
            font_size=25,
        )
        im = to_pil_image(box.detach())

    save_image_with_title(title, im, output_path, model_name, image_path)

#### **extract_number function**
This function returns a sorted list of the files in a directory by extracting the number in the file names.
* *directory_name*: name of directory with the files you need sorted. Note: this only works if the files already have some unique numerical ID

In [84]:
def extract_number(directory_name):
    match = re.findall(r"\d+", directory_name)
    # if match:
    return int(match[0])
    # return float("inf")


# Creating a sorted list of the filenames from my images dir
sorted_filenames = sorted(os.listdir("images"), key=extract_number)

This code goes through the sorted image file names and saves annotated images of the predictions accross each model.

In [85]:
for image in sorted_filenames[:5]:
    path = "images/" + image

    object_detection(
        fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights, path
    )
    object_detection(fcos_resnet50_fpn, FCOS_ResNet50_FPN_Weights, path)

    object_detection(retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights, path)

    object_detection(ssd300_vgg16, SSD300_VGG16_Weights, path)

    object_detection(
        ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights, path
    )

Model Output saved to evaluation/outputs/fasterrcnn_resnet50_fpn_v2/image_00650358.jpg
Model Output saved to evaluation/outputs/fcos_resnet50_fpn/image_00650358.jpg
Model Output saved to evaluation/outputs/retinanet_resnet50_fpn_v2/image_00650358.jpg
Model Output saved to evaluation/outputs/ssd300_vgg16/image_00650358.jpg
Model Output saved to evaluation/outputs/ssdlite320_mobilenet_v3_large/image_00650358.jpg
Model Output saved to evaluation/outputs/fasterrcnn_resnet50_fpn_v2/image_00650363.jpg
Model Output saved to evaluation/outputs/fcos_resnet50_fpn/image_00650363.jpg
Model Output saved to evaluation/outputs/retinanet_resnet50_fpn_v2/image_00650363.jpg
Model Output saved to evaluation/outputs/ssd300_vgg16/image_00650363.jpg
Model Output saved to evaluation/outputs/ssdlite320_mobilenet_v3_large/image_00650363.jpg
Model Output saved to evaluation/outputs/fasterrcnn_resnet50_fpn_v2/image_00650921.jpg
Model Output saved to evaluation/outputs/fcos_resnet50_fpn/image_00650921.jpg
Model O

this code generates and saves a grid comparision image of the models.

In [86]:
# Extra Paths for the Sake of Documentation:
extra_paths = [
    "images/image_2016887160.jpg",
    "../workflow/image-collection-output/image_2017879462.jpg",
    "../workflow/image-collection-output/image_2021643419.jpg",
]

for path in extra_paths:
    object_detection(
        fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights, path
    )
    object_detection(fcos_resnet50_fpn, FCOS_ResNet50_FPN_Weights, path)

    object_detection(retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights, path)

    object_detection(ssd300_vgg16, SSD300_VGG16_Weights, path)

    object_detection(
        ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights, path
    )

Model Output saved to evaluation/outputs/fasterrcnn_resnet50_fpn_v2/image_2016887160.jpg
Model Output saved to evaluation/outputs/fcos_resnet50_fpn/image_2016887160.jpg
Model Output saved to evaluation/outputs/retinanet_resnet50_fpn_v2/image_2016887160.jpg
Model Output saved to evaluation/outputs/ssd300_vgg16/image_2016887160.jpg
Model Output saved to evaluation/outputs/ssdlite320_mobilenet_v3_large/image_2016887160.jpg
Model Output saved to evaluation/outputs/fasterrcnn_resnet50_fpn_v2/image_2017879462.jpg
Model Output saved to evaluation/outputs/fcos_resnet50_fpn/image_2017879462.jpg
Model Output saved to evaluation/outputs/retinanet_resnet50_fpn_v2/image_2017879462.jpg
Model Output saved to evaluation/outputs/ssd300_vgg16/image_2017879462.jpg
Model Output saved to evaluation/outputs/ssdlite320_mobilenet_v3_large/image_2017879462.jpg
Model Output saved to evaluation/outputs/fasterrcnn_resnet50_fpn_v2/image_2021643419.jpg
Model Output saved to evaluation/outputs/fcos_resnet50_fpn/imag

In [88]:
# read images from computer
eval_outputs_dir = "../early_work/evaluation/outputs/"

folders = os.listdir(eval_outputs_dir)
# image
grid_number = len(os.listdir(eval_outputs_dir + "fasterrcnn_resnet50_fpn_v2"))
for grid_image in range(grid_number):
    image_vars = []
    for i, folder in enumerate(os.listdir(eval_outputs_dir)):
        if folder != ".DS_Store":
            sub_name = eval_outputs_dir + folder
            sub_directory = os.listdir(sub_name)
            # print(sub_directory[0])
            file_name = sub_name + "/" + sub_directory[grid_image]
            image_vars.append(read_image(file_name))

        base_name = os.path.basename(file_name)
        Grid = make_grid(image_vars)
        img = torchvision.transforms.ToPILImage()(Grid)

        comparison_dir = "evaluation/" + "model_comparisons"
        if not os.path.exists(comparison_dir):
            os.makedirs(comparison_dir)

    panorama_name = comparison_dir + "/comparison_" + base_name
    img.save(panorama_name, "JPEG")
    print(f"Model Comparison saved as {panorama_name}")

Model Comparison saved as evaluation/model_comparisons/comparison_image_2021643419.jpg
Model Comparison saved as evaluation/model_comparisons/comparison_image_2016887160.jpg
Model Comparison saved as evaluation/model_comparisons/comparison_image_00650923.jpg
Model Comparison saved as evaluation/model_comparisons/comparison_image_00650921.jpg
Model Comparison saved as evaluation/model_comparisons/comparison_image_2017879462.jpg
Model Comparison saved as evaluation/model_comparisons/comparison_image_00650930.jpg
Model Comparison saved as evaluation/model_comparisons/comparison_image_00650363.jpg
Model Comparison saved as evaluation/model_comparisons/comparison_image_00650358.jpg
